# Load libraries and set device up

In [1]:
import seaborn           as sns
import numpy             as np
import matplotlib.pyplot as plt
import torch
import json

import libraries.model   as clm
import libraries.dataset as cld

# Checking if pytorch can run in GPU, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

sns.set_theme()

# Define parameters

In [2]:
target = 'EPA'
folder = 'MP-EPA-deep'

r_folder = f'models/{folder}'
t_folder = r_folder
#t_folder = f'datasets/bandgap-EPA'
model_name = f'{r_folder}/{target}-model.pt'

# Generate or load graph database for training

Reference dataset

In [3]:
r_dataset_name_std      = f'{r_folder}/train_dataset_std.pt'
r_labels_name_std       = f'{r_folder}/train_labels_std.pt'
r_std_parameters_name   = f'{r_folder}/standardized_parameters.json'  # Parameters for rescaling the predictions
r_uncertainty_data_name = f'{r_folder}/uncertainty_data.json'

# Load the standardized dataset, with corresponding labels and parameters
r_dataset_std = torch.load(r_dataset_name_std)
r_labels_std  = torch.load(r_labels_name_std)

# Load the data from the JSON file
with open(r_uncertainty_data_name, 'r') as json_file:
    r_uncertainty_data = json.load(json_file)

# Load the data from the JSON file
with open(r_std_parameters_name, 'r') as json_file:
    numpy_dict = json.load(json_file)

# Convert torch tensors to numpy arrays
r_std_parameters = {}
for key, value in numpy_dict.items():
    try:
        r_std_parameters[key] = torch.tensor(value)
    except:
        r_std_parameters[key] = value

# Load the data from the JSON file
with open(f'{r_folder}/dataset_parameters.json', 'r') as json_file:
    r_dataset_parameters = json.load(json_file)

# Determine the index for the desired property
target_idx = [idx for idx, t in enumerate(r_dataset_parameters['target']) if t == target][0]
r_std_parameters['target_mean'] = r_std_parameters['target_mean'][target_idx]
r_std_parameters['target_std']  = r_std_parameters['target_std'][target_idx]

# Defining target factor
target_factor = r_std_parameters['target_std'] / r_std_parameters['scale']

/tmp/ipykernel_978043/1521842984.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  r_dataset_std = torch.load(r_dataset_name_std)
/tmp/ipykernel_978043/1521842984.py:8: Fu

Prediciton dataset

In [4]:
t_dataset_name = f'{t_folder}/test_dataset_std.pt'

# Load the standardized dataset, with corresponding labels and parameters
t_dataset_std = torch.load(t_dataset_name)

target_mean = r_std_parameters['target_mean']

ground_truths = [(data.y * target_factor + target_mean).item() for data in t_dataset_std]

/tmp/ipykernel_978043/3046611322.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  t_dataset_std = torch.load(t_dataset_name)


p_dataset_name = f'{t_folder}/dataset.pt'

# Load the standardized dataset, with corresponding labels and parameters
p_dataset = torch.load(p_dataset_name)

# Standardize the dataset with
p_dataset_std = cld.standarize_dataset_from_keys(p_dataset, r_std_parameters)

# Select the target we want
ground_truths = [data.y[target_idx].item() for data in p_dataset]
del p_dataset

# Load model and make predictions

In [ ]:
# Load Graph Neural Network model
model = clm.GCNN(features_channels=r_dataset_std[0].num_node_features, pdropout=0)

# Moving model to device
model = model.to(device)

# Load Graph Neural Network model
model.load_state_dict(torch.load(model_name, map_location=torch.device(device)))
model.eval()  # Evaluation mode, drops out dropouts

# Compute predictions
predictions, uncertainties, are_interpolated = clm.make_predictions(r_dataset_std, r_labels_std,
                                                                    t_dataset_std, model,
                                                                    r_std_parameters, r_uncertainty_data)

/tmp/ipykernel_978043/3839645800.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_name, map_location=torch.device(device)))


# Plot predictions

In [ ]:
stack = np.concatenate([ground_truths, predictions])

_min_ = np.min(stack)
_max_ = np.max(stack)

xpred = ground_truths[are_interpolated]
ypred = predictions[are_interpolated]
yerr  = np.abs(uncertainties[are_interpolated])
plt.errorbar(xpred, ypred, yerr=yerr,
             fmt='og', label='Test, interp')

xpred = ground_truths[~are_interpolated]
ypred = predictions[~are_interpolated]
yerr  = np.abs(ground_truths[~are_interpolated])
plt.errorbar(xpred, ypred, yerr=yerr,
             fmt='ob', label='Test, extrap')

plt.plot([_min_, _max_], [_min_, _max_], '-r')

plt.xlabel(r'Computed $E_0$ (eV/atom)')
plt.ylabel(r'Predicted $E_0$ (eV/atom)')
plt.legend(loc='best')
plt.savefig(f'{t_folder}/UQ-prediction-comparison.pdf', dpi=50, bbox_inches='tight')
plt.show()

In [ ]:
gt_uncertainties   = np.abs(ground_truths - predictions)
pred_uncertainties = np.abs(uncertainties)

stack = np.concatenate([gt_uncertainties, pred_uncertainties])

_min_ = np.min(stack)
_max_ = np.max(stack)

plt.plot(gt_uncertainties, pred_uncertainties,  '.', label='Test')
plt.plot([_min_, _max_], [_min_, _max_], '-r')
plt.xlabel(r'Computed $s_{E_0}$ (eV/atom)')
plt.ylabel(r'Predicted $s_{E_0}$ (eV/atom)')
plt.legend(loc='best')
plt.savefig(f'{t_folder}/UQ-values-comparison.pdf', dpi=50, bbox_inches='tight')
plt.show()